# How can the company improve collaboration?

## 📖 Background
You work in the analytics department of a multinational company, and the head of HR wants your help mapping out the company's employee network using message data. 

They plan to use the network map to understand interdepartmental dynamics better and explore how the company shares information. The ultimate goal of this project is to think of ways to improve collaboration throughout the company. 

## 💾 The data

The company has six months of information on inter-employee communication. For privacy reasons, only sender, receiver, and message length information are available [(source)](https://snap.stanford.edu/data/CollegeMsg.html). 

#### Messages has information on the sender, receiver, and time.
- "sender" - represents the employee id of the employee sending the message.
- "receiver" - represents the employee id of the employee receiving the message.
- "timestamp" - the date of the message.
- "message_length" - the length in words of the message.

#### Employees has information on each employee;
- "id" - represents the employee id of the employee.
- "department" - is the department within the company. 
- "location" - is the country where the employee lives.
- "age" - is the age of the employee.

#### Insights to accomplish:

- Which departments are the most/least active?
- Which employee has the most connections?
- Identify the most influential departments and employees.
- Using the network analysis, in which departments would you recommend the HR team focus to boost collaboration?

_**Acknowledgments:** Pietro Panzarasa, Tore Opsahl, and Kathleen M. Carley. "Patterns and dynamics of users' behavior and interaction: Network analysis of an online community." Journal of the American Society for Information Science and Technology 60.5 (2009): 911-932._

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import Window

In [3]:
# Load Data Function
def loadDf(fileName):
    dt = spark.read.format('delta').options(header='true').load(fileName)
    return dt

In [4]:
# employees
dtEmployees = loadDf("dbfs:/user/hive/warehouse/employees")

dtEmployees.show(n=40)

+---+-----------+--------+---+
| id| department|location|age|
+---+-----------+--------+---+
|  3| Operations|      US| 33|
|  6|      Sales|      UK| 50|
|  8|         IT|  Brasil| 54|
|  9|      Admin|      UK| 32|
| 12| Operations|  Brasil| 51|
| 19|  Marketing|      US| 50|
| 23|      Sales|  Brasil| 39|
| 26| Operations|  France| 32|
| 27|      Sales|  France| 58|
| 29|      Admin|  France| 33|
| 30|      Sales| Germany| 54|
| 32|      Sales|      UK| 54|
| 34|  Marketing| Germany| 46|
| 36|      Admin|      US| 39|
| 41|         IT|      UK| 56|
| 42|      Admin|      US| 33|
| 44|      Sales| Germany| 54|
| 47|      Sales|  Brasil| 22|
| 48|         IT|  France| 34|
| 50|      Admin|      US| 33|
| 51|      Admin|      UK| 30|
| 52|         IT|      UK| 51|
| 53| Operations|      US| 51|
| 58|      Sales| Germany| 40|
| 60|Engineering| Germany| 38|
| 62| Operations|  France| 34|
| 63|      Sales|  France| 38|
| 67|         IT|  France| 42|
| 68|      Admin|  France| 50|
| 70| Op

In [5]:
# employees
dtMessages = loadDf("dbfs:/user/hive/warehouse/messages")

dtMessages.show(40)

+------+--------+-------------------+--------------+
|sender|receiver|          timestamp|message_length|
+------+--------+-------------------+--------------+
|    79|      48|2021-06-02 05:41:34|            88|
|    79|      63|2021-06-02 05:42:15|            72|
|    79|      58|2021-06-02 05:44:24|            86|
|    79|      70|2021-06-02 05:49:07|            26|
|    79|     109|2021-06-02 19:51:47|            73|
|    79|      58|2021-06-03 01:12:11|            37|
|   144|      99|2021-06-03 09:54:41|            33|
|   144|     105|2021-06-03 09:57:02|            80|
|   144|     121|2021-06-03 09:59:16|            13|
|   177|      32|2021-06-04 07:50:33|            32|
|   177|     180|2021-06-04 07:59:18|            85|
|   177|     180|2021-06-04 08:00:22|            75|
|   177|      32|2021-06-04 08:01:10|            30|
|   177|      32|2021-06-04 08:08:34|            26|
|   177|      32|2021-06-04 08:10:44|            44|
|   177|      32|2021-06-04 08:17:37|         

#### Which departments are the most/least active?
#### Identify the most influential departments and employees.

In [7]:
# Total Messages Sent by department
dtTMS = dtEmployees.join(dtMessages, dtEmployees.id == dtMessages.sender) \
    .groupBy(dtEmployees.department) \
    .agg(count(dtMessages.sender).alias("Total Messages Sent")) \
    .orderBy("Total Messages Sent",ascending=False)

dtTMS.show()

+-----------+-------------------+
| department|Total Messages Sent|
+-----------+-------------------+
|      Sales|               1551|
| Operations|               1013|
|      Admin|                857|
|         IT|                 49|
|Engineering|                 26|
|  Marketing|                 16|
+-----------+-------------------+



In [8]:
# Total Messages Sent by department
dtTMS = dtEmployees.join(dtMessages, dtEmployees.id == dtMessages.sender) \
    .groupBy(dtEmployees.department) \
    .agg(count(dtMessages.sender).alias("Total Messages Sent")) \
    .orderBy("Total Messages Sent",ascending=False)

dtTMS.show()

+-----------+-------------------+
| department|Total Messages Sent|
+-----------+-------------------+
|      Sales|               1551|
| Operations|               1013|
|      Admin|                857|
|         IT|                 49|
|Engineering|                 26|
|  Marketing|                 16|
+-----------+-------------------+



In [9]:
# Total Messages Sent by department
dtTMS = dtEmployees.join(dtMessages, dtEmployees.id == dtMessages.sender) \
    .groupBy(dtEmployees.department) \
    .agg(count(dtMessages.sender).alias("Total Messages Sent")) \
    .orderBy("Total Messages Sent",ascending=False)

dtTMS.show()

+-----------+-------------------+
| department|Total Messages Sent|
+-----------+-------------------+
|      Sales|               1551|
| Operations|               1013|
|      Admin|                857|
|         IT|                 49|
|Engineering|                 26|
|  Marketing|                 16|
+-----------+-------------------+



In [10]:
# Total Messages Received by department
dtTMR = dtEmployees.join(dtMessages, dtEmployees.id == dtMessages.receiver) \
    .groupBy(dtEmployees.department) \
    .agg(count(dtMessages.receiver).alias("Total Messages Received")) \
    .orderBy("Total Messages Received",ascending=False)

dtTMR.show()


+-----------+-----------------------+
| department|Total Messages Received|
+-----------+-----------------------+
|      Sales|                   1229|
| Operations|                    845|
|      Admin|                    797|
|Engineering|                    252|
|         IT|                    249|
|  Marketing|                    140|
+-----------+-----------------------+



In [11]:
# Total Messages ( Activity ) by department
dtTM = dtTMS.unionAll(dtTMR)

dtTMF = dtTM\
        .groupBy("department")\
        .agg(sum(col("Total Messages Sent")).alias("Total Messages"))\
        .orderBy("Total Messages",ascending=False)\
        .select("department",(col("Total Messages")))
       
dtTMF.show()

+-----------+--------------+
| department|Total Messages|
+-----------+--------------+
|      Sales|          2780|
| Operations|          1858|
|      Admin|          1654|
|         IT|           298|
|Engineering|           278|
|  Marketing|           156|
+-----------+--------------+



#### Answer: The most active department is the Sales and the least is the Marketing

#### Which employee has the most connections?

In [14]:
# Employees with more Emails sent to different employees
dtTmsId = dtEmployees.join(dtMessages, dtEmployees.id == dtMessages.sender) \
    .groupBy(dtEmployees.department,dtEmployees.id) \
    .agg(count(dtMessages.sender).alias("Emails_Sent"),
         countDistinct(dtMessages.receiver).alias("Unique_Emails_Adresses_Sent")) \
    .orderBy(col("Unique_Emails_Adresses_Sent").desc(),(col("Emails_sent").desc()))
    
dtTmsId.show()

+----------+----+-----------+---------------------------+
|department|  id|Emails_Sent|Unique_Emails_Adresses_Sent|
+----------+----+-----------+---------------------------+
|Operations| 598|        187|                         77|
|     Sales| 144|        221|                         75|
|     Sales| 128|        266|                         71|
|     Admin| 605|        459|                         68|
|Operations| 586|        180|                         64|
|     Sales| 389|        196|                         58|
|Operations| 317|        184|                         51|
|     Admin| 509|        216|                         50|
|     Sales| 337|         91|                         39|
|Operations| 260|         77|                         33|
|     Sales| 725|        137|                         32|
|     Sales| 422|         79|                         31|
|     Sales| 483|        169|                         30|
|Operations|1142|         44|                         24|
|     Sales|10

In [15]:
# Employees with more Emails received from different employees
dtTmrId = dtEmployees.join(dtMessages, dtEmployees.id == dtMessages.receiver) \
    .groupBy(dtEmployees.department,dtEmployees.id) \
    .agg(count(dtMessages.receiver).alias("Emails_Received"),
         countDistinct(dtMessages.sender).alias("Unique_Emails_Adresses_Received")) \
    .orderBy(col("Unique_Emails_Adresses_Received").desc(),(col("Emails_Received").desc()))
    
dtTmrId.show()

+----------+----+---------------+-------------------------------+
|department|  id|Emails_Received|Unique_Emails_Adresses_Received|
+----------+----+---------------+-------------------------------+
|     Admin| 194|             21|                             13|
|     Sales|  32|             47|                             11|
|     Admin| 249|             18|                              9|
|     Sales| 103|             40|                              8|
|     Admin|  42|             25|                              8|
|     Admin|1283|             11|                              8|
|Operations| 263|             39|                              7|
|     Sales| 400|             23|                              7|
|     Admin| 840|             16|                              7|
|Operations| 598|             11|                              7|
|        IT| 105|             11|                              7|
|     Sales| 308|             51|                              6|
|     Sale

In [16]:
# The Employees Connections
dtTmFId = dtTmrId.join(dtTmsId, dtTmsId.id == dtTmrId.id)\
        .groupBy(dtTmrId.id, dtTmrId.department,\
                dtTmsId.Emails_Sent,dtTmsId.Unique_Emails_Adresses_Sent,\
                dtTmrId.Emails_Received,dtTmrId.Unique_Emails_Adresses_Received) \
        .agg((dtTmsId.Unique_Emails_Adresses_Sent+dtTmrId.Unique_Emails_Adresses_Received).alias("Connections"))\
        .select(dtTmrId.id, dtTmrId.department,\
                dtTmsId.Emails_Sent,dtTmsId.Unique_Emails_Adresses_Sent,\
                dtTmrId.Emails_Received,dtTmrId.Unique_Emails_Adresses_Received,\
                col("Connections"))\
        .orderBy(col("Connections").desc())
        
dtTmFId.display()

id,department,Emails_Sent,Unique_Emails_Adresses_Sent,Emails_Received,Unique_Emails_Adresses_Received,Connections
598,Operations,187,77,11,7,84
144,Sales,221,75,8,3,78
128,Sales,266,71,5,4,75
605,Admin,459,68,37,3,71
586,Operations,180,64,3,1,65
389,Sales,196,58,22,4,62
317,Operations,184,51,26,4,55
509,Admin,216,50,14,5,55
337,Sales,91,39,22,3,42
260,Operations,77,33,10,6,39


#### The employee that has the most connections (Sum of sent and received emails for different employees) is 598.

#### Identify the most influential departments and employees.

In [19]:
#Most influential departments (top 3)
dtMid = dtTmrId.join(dtTmsId, dtTmsId.id == dtTmrId.id)\
        .groupBy(dtTmrId.department)\
        .agg(sum(dtTmsId.Unique_Emails_Adresses_Sent+dtTmrId.Unique_Emails_Adresses_Received).alias("Connections"))\
        .orderBy(col("Connections").desc())\
        .limit(3)

dtMid.display()

department,Connections
Sales,369
Operations,355
Admin,190


In [20]:
#Most influential employees (top 3)
dtMie = dtTmrId.join(dtTmsId, dtTmsId.id == dtTmrId.id)\
        .groupBy(dtTmrId.id)\
        .agg(sum(dtTmsId.Unique_Emails_Adresses_Sent+dtTmrId.Unique_Emails_Adresses_Received).alias("Connections"))\
        .orderBy(col("Connections").desc())\
        .limit(3)

dtMie.display()

id,Connections
598,84
144,78
128,75


#### The top 3 employees that have more influence on the company are 598,144,128.
#### The top 3 departments that have more influence on the company are Sales, Operations and Admin.

In [22]:
#Less influential departments (top 3)
dtMid = dtTmrId.join(dtTmsId, dtTmsId.id == dtTmrId.id)\
        .groupBy(dtTmrId.department)\
        .agg(sum(dtTmsId.Unique_Emails_Adresses_Sent+dtTmrId.Unique_Emails_Adresses_Received).alias("Connections"))\
        .orderBy(col("Connections").asc())\
        .limit(3)

dtMid.display()

department,Connections
Engineering,7
Marketing,9
IT,19


#### The departments that I recommend the HR team focus to boost collaboration are Enginnering,Marketing and IT.